<a href="https://colab.research.google.com/github/Sirmuchai/Augmenting-Pandas-with-SQLite/blob/main/Week_5_Project_1_Augmenting_Pandas_with_SQLite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project Notebook: Augmenting Pandas with SQLite

## Question 1: Introduction

In this session, we explored a few different ways to work with larger datasets in pandas. In this guided project, we'll practice using some of the techniques we learned to analyze startup investments from Crunchbase.com.

Every year, thousands of startup companies raise financing from investors. Each time a startup raises money, we refer to the event as a fundraising round. Crunchbase is a website that crowdsources information on the fundraising rounds of many startups. The Crunchbase user community submits, edits, and maintains most of the information in Crunchbase.

In return, Crunchbase makes the data available through a Web application and a fee-based API. Before Crunchbase switched to the paid API model, multiple groups crawled the site and released the data online. Because the information on the startups and their fundraising rounds is always changing, the data set we'll be using isn't completely up to date.

Throughout this project, we'll practice working with different memory constraints. In this step, let's assume we only have 10 megabytes of available memory. While crunchbase-investments.csv (https://bit.ly/3BPcobU) consumes 10.3 megabytes of disk space, we know from earlier lessons that pandas often requires 4 to 6 times amount of space in memory as the file does on disk (especially when there's many string columns).


**Tasks**

* Because the data set contains over 50,000 rows, you'll need to read the data set into dataframes using 5,000 row chunks to ensure that each chunk consumes much less than 10 megabytes of memory.
* Across all of the chunks, become familiar with:
1. Each column's missing value counts.
2. Each column's memory footprint.
3. The total memory footprint of all of the chunks combined.
4. Which column(s) we can drop because they aren't useful for analysis.



In [1]:
# Your code goes here
# Dataset URL = https://bit.ly/3BPcobU
#

import pandas as pd

# Define the file path
file_path = "https://bit.ly/3BPcobU"

# Define the chunk size
chunk_size = 5000

# Create an empty list to store the chunks
chunks = []

# Loop over the file in chunks
for chunk in pd.read_csv(file_path, encoding='ISO-8859-1', chunksize=chunk_size):
    chunks.append(chunk)

# Loop over the chunks and display the column information
for chunk in chunks:
    print(chunk.info(memory_usage='deep'))

# Calculate the total memory usage of all the chunks
total_memory_usage = sum(chunk.memory_usage(deep=True).sum() for chunk in chunks)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 20 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   company_permalink       5000 non-null   object 
 1   company_name            5000 non-null   object 
 2   company_category_code   4948 non-null   object 
 3   company_country_code    5000 non-null   object 
 4   company_state_code      4947 non-null   object 
 5   company_region          5000 non-null   object 
 6   company_city            4936 non-null   object 
 7   investor_permalink      5000 non-null   object 
 8   investor_name           5000 non-null   object 
 9   investor_category_code  2443 non-null   object 
 10  investor_country_code   4222 non-null   object 
 11  investor_state_code     3629 non-null   object 
 12  investor_region         5000 non-null   object 
 13  investor_city           4100 non-null   object 
 14  funding_round_type      5000 non-null   

## Question 2: Selecting Data Types

Now that we have a good sense of the missing values, let's get familiar with the column types before adding the data into SQLite.

**Tasks**

* Identify the types for each column.
* Identify the numeric columns we can represent using more space efficient types.
For text columns:
* Analyze the unique value counts across all of the chunks to see if we can convert them to a numeric type.
* See if we clean clean any text columns and separate them into multiple numeric columns without adding any overhead when querying.
* Make your changes to the code from the last step so that the overall memory the data consumes stays under 10 megabytes.


In [2]:
# Your code goes here

# Loop over the chunks and display the data types
for chunk in chunks:
    print(chunk.dtypes)


company_permalink          object
company_name               object
company_category_code      object
company_country_code       object
company_state_code         object
company_region             object
company_city               object
investor_permalink         object
investor_name              object
investor_category_code     object
investor_country_code      object
investor_state_code        object
investor_region            object
investor_city              object
funding_round_type         object
funded_at                  object
funded_month               object
funded_quarter             object
funded_year                 int64
raised_amount_usd         float64
dtype: object
company_permalink          object
company_name               object
company_category_code      object
company_country_code       object
company_state_code         object
company_region             object
company_city               object
investor_permalink         object
investor_name              object


In [3]:
# Concatenate the chunks into a single DataFrame
df = pd.concat(chunks)

# Loop over the columns and display the unique value counts
for col in df.columns:
    print(f'{col}: {df[col].nunique()} unique values')


company_permalink: 11573 unique values
company_name: 11573 unique values
company_category_code: 43 unique values
company_country_code: 2 unique values
company_state_code: 50 unique values
company_region: 546 unique values
company_city: 1229 unique values
investor_permalink: 10552 unique values
investor_name: 10465 unique values
investor_category_code: 33 unique values
investor_country_code: 72 unique values
investor_state_code: 50 unique values
investor_region: 585 unique values
investor_city: 990 unique values
funding_round_type: 9 unique values
funded_at: 2808 unique values
funded_month: 192 unique values
funded_quarter: 72 unique values
funded_year: 20 unique values
raised_amount_usd: 1458 unique values


In [4]:
# Define a function to clean and split the company_category_code column
def clean_category_list(df):
    # Replace missing values with an empty string
    df['company_category_code'] = df['company_category_code'].fillna('')
    # Split the column into multiple columns using the vertical bar as a delimiter
    split_cols = df['company_category_code'].str.split('|', expand=True)
    # Rename the columns
    split_cols.columns = [f'category_{i}' for i in range(split_cols.shape[1])]
    # Convert the new columns to categorical types
    for col in split_cols.columns:
        split_cols[col] = split_cols[col].astype('category')
    # Concatenate the new columns with the original DataFrame
    return pd.concat([df, split_cols], axis=1)

# Loop over the chunks, clean the category list column, and display the memory usage
memory_usage = 0
for i, chunk in enumerate(chunks):
    chunk = clean_category_list(chunk)
    memory_usage += chunk.memory_usage(deep=True).sum()
    print(f'Chunk {i+1} memory usage: {memory_usage/1e6:.2f} MB')


Chunk 1 memory usage: 5.86 MB
Chunk 2 memory usage: 11.67 MB
Chunk 3 memory usage: 17.48 MB
Chunk 4 memory usage: 23.29 MB
Chunk 5 memory usage: 29.09 MB
Chunk 6 memory usage: 34.93 MB
Chunk 7 memory usage: 40.74 MB
Chunk 8 memory usage: 46.52 MB
Chunk 9 memory usage: 52.19 MB
Chunk 10 memory usage: 57.07 MB
Chunk 11 memory usage: 59.87 MB


## Question 3: Loading Chunks Into SQLite

Now we're in good shape to start exploring and analyzing the data. The next step is to load each chunk into a table in a SQLite database so we can query the full data set.

**Tasks**

1. Create and connect to a new SQLite database file.
2. Expand on the existing chunk processing code to export each chunk to a new table in the SQLite database.
3. Query the table and make sure the data types match up to what you had in mind for each column.

In [18]:
# Your code goes here
from google.colab import drive
import sqlite3

# Mount Google Drive
drive.mount('/content/drive')

# Create a new SQLite database file in a mounted directory
conn = sqlite3.connect('/content/drive/MyDrive/crunchbase.db')

# Define the table name prefix
table_name= 'investments'

# Loop through the CSV file in chunks
for chunk in chunks:
    # Define the table name for this chunk
    #table_name = table_name_prefix + str(i)
    
    # Export the chunk to the SQLite database
    chunk.to_sql(table_name, conn, if_exists='replace', index=False)
    



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [19]:
# Get information about the columns in the first chunk table
cursor = conn.execute('PRAGMA table_info(investments)')
for row in cursor:
    print(row)



(0, 'company_permalink', 'TEXT', 0, None, 0)
(1, 'company_name', 'TEXT', 0, None, 0)
(2, 'company_category_code', 'TEXT', 0, None, 0)
(3, 'company_country_code', 'TEXT', 0, None, 0)
(4, 'company_state_code', 'TEXT', 0, None, 0)
(5, 'company_region', 'TEXT', 0, None, 0)
(6, 'company_city', 'TEXT', 0, None, 0)
(7, 'investor_permalink', 'TEXT', 0, None, 0)
(8, 'investor_name', 'TEXT', 0, None, 0)
(9, 'investor_category_code', 'REAL', 0, None, 0)
(10, 'investor_country_code', 'REAL', 0, None, 0)
(11, 'investor_state_code', 'REAL', 0, None, 0)
(12, 'investor_region', 'TEXT', 0, None, 0)
(13, 'investor_city', 'REAL', 0, None, 0)
(14, 'funding_round_type', 'TEXT', 0, None, 0)
(15, 'funded_at', 'TEXT', 0, None, 0)
(16, 'funded_month', 'TEXT', 0, None, 0)
(17, 'funded_quarter', 'TEXT', 0, None, 0)
(18, 'funded_year', 'INTEGER', 0, None, 0)
(19, 'raised_amount_usd', 'REAL', 0, None, 0)


## Question 4: Next Steps

Now that the data is in SQLite, we can use the pandas SQLite workflow we learned in the last lesson to explore and analyze startup investments. Remember that each row isn't a unique company, but a unique investment from a single investor. This means that many startups will span multiple rows.

Use the pandas SQLite workflow to answer the following questions:

* What proportion of the total amount of funds did the top 10% raise? What about the top 1%? Compare these values to the proportions the bottom 10% and bottom 1% raised.
* Which category of company attracted the most investments?
* Which investor contributed the most money (across all startups)?
* Which investors contributed the most money per startup?
* Which funding round was the most popular? Which was the least popular?

Here are some ideas for further exploration:

* Repeat the tasks in this project using stricter memory constraints (under 1 megabyte).
* Clean and analyze the other Crunchbase data sets from the same GitHub repo.
* Understand which columns the data sets share, and how the data sets are linked.
* Create a relational database design that links the data sets together and reduces the overall disk space the database file consumes.

Use pandas to populate each table in the database, create the appropriate indexes, and so on.

In [20]:
# Your code goes here


# Define the SQL query
query = '''
SELECT raised_amount_usd,
       SUM(raised_amount_usd) OVER (ORDER BY raised_amount_usd DESC) AS cumsum
FROM investments
'''

# Read the results of the query into a DataFrame
df = pd.read_sql_query(query, conn)

# Calculate the total amount of funds raised
total_raised = df['raised_amount_usd'].sum()

In [21]:
# Calculate the amount of funds raised by the top and bottom 10% and 1% of companies
top_10_percent_raised = df.loc[df['cumsum'] <= 0.1 * total_raised, 'raised_amount_usd'].sum()
bottom_10_percent_raised = df.loc[df['cumsum'] >= 0.9 * total_raised, 'raised_amount_usd'].sum()
top_1_percent_raised = df.loc[df['cumsum'] <= 0.01 * total_raised, 'raised_amount_usd'].sum()
bottom_1_percent_raised = df.loc[df['cumsum'] >= 0.99 * total_raised, 'raised_amount_usd'].sum()

# Calculate the proportion of the total amount of funds raised by the top and bottom 10% and 1% of companies
top_10_percent_proportion = top_10_percent_raised / total_raised
bottom_10_percent_proportion = bottom_10_percent_raised / total_raised
top_1_percent_proportion = top_1_percent_raised / total_raised
bottom_1_percent_proportion = bottom_1_percent_raised / total_raised

# Print the results
print('Top 10% proportion: {:.2%}'.format(top_10_percent_proportion))
print('Bottom 10% proportion: {:.2%}'.format(bottom_10_percent_proportion))
print('Top 1% proportion: {:.2%}'.format(top_1_percent_proportion))
print('Bottom 1% proportion: {:.2%}'.format(bottom_1_percent_proportion))

Top 10% proportion: 9.57%
Bottom 10% proportion: 12.19%
Top 1% proportion: 0.00%
Bottom 1% proportion: 1.33%


### Which category of company attracted the most investments?

In [22]:
# Query the data from the investments table
data = pd.read_sql_query('SELECT * FROM investments', conn)

# Group the data by category and count the number of investments for each category
investments_by_category = data.groupby('company_category_code')['raised_amount_usd'].count()

# Find the category with the highest number of investments
most_investments_category = investments_by_category.idxmax()

print(f"The category with the most investments is {most_investments_category}")

The category with the most investments is web


**The category with the most investments is software**

### Which investor contributed the most money (across all startups)?

In [23]:
# Query the data from the investments table
data = pd.read_sql_query('SELECT * FROM investments', conn)

# Group the data by investor and sum the amount of money invested by each investor
investments_by_investor = data.groupby('investor_name')['raised_amount_usd'].sum()

# Find the investor who contributed the most money
most_investments_investor = investments_by_investor.idxmax()

print(f"The investor who contributed the most money is {most_investments_investor}")

The investor who contributed the most money is Peter Thiel


**The investor who contributed the most money is Peter Thiel**

### Which investors contributed the most money per startup?

In [24]:
query = "SELECT * FROM investments"
df = pd.read_sql_query(query, conn)

# group the data by startup name and investor name, and sum the investments
grouped = df.groupby(['company_name', 'investor_name'], as_index=False)['raised_amount_usd'].sum()

# find the investor that contributed the most money to each startup
idx = grouped.groupby('company_name')['raised_amount_usd'].transform(max) == grouped['raised_amount_usd']
result = grouped[idx]

# print the result
print(result[['company_name', 'investor_name', 'raised_amount_usd']].sort_values('raised_amount_usd', ascending=False))

                         company_name   investor_name  raised_amount_usd
1136          Legendary Entertainment     Peter Thiel        275000000.0
1001             Intrexon Corporation  Randal J. Kirk        150000000.0
2124                  T5 Data Centers   Raymond James        113000000.0
1601  Pocket Communications Northeast     Paul Posner        100000000.0
408                        CommonBond     Tom Kalaris        100000000.0
...                               ...             ...                ...
344                            ChatID   Patrick Keane                0.0
345                            ChatID      Ralph Mack                0.0
346                            ChatID  Rob Wiesenthal                0.0
347                            ChatID   Zach Weinberg                0.0
999            Internet Marketing Inc       Ryan Born                0.0

[2383 rows x 3 columns]


### Which funding round was the most popular? Which was the least popular?

In [28]:
# Query the number of investments in each funding round
query = '''
        SELECT funding_round_type, COUNT(*) as num_investments
        FROM investments
        GROUP BY funding_round_type
        ORDER BY num_investments DESC
        '''

round_counts = pd.read_sql_query(query, conn)
print(round_counts)

  funding_round_type  num_investments
0              angel             1342
1           series-a             1037
2            venture              215
3           series-b              148
4          series-c+               88
5              other               30
6     private-equity               10
